# IMPORTS

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
# data
import numpy as np
import pandas as pd

# graphics
import plotly.express as px
import matplotlib.pyplot as plt

# stats
from pandas.plotting import autocorrelation_plot # test for seasonality
from pandas.plotting import lag_plot # test for autocorrelation
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy import signal

# src
from src.utilities import from_pickle

# PATHS & NAMES

In [3]:
RAW_FILENAME = "dataset.parquet"
RAWFILE_FOLDER = "../data/raw"
RAW_FILEPATH = f"{RAWFILE_FOLDER}/{RAW_FILENAME}"

INTERIM_FILENAME = "transformed.pkl"
INTERIM_FOLDER = "../data/interim"
INTERIM_FILEPATH = f"{INTERIM_FOLDER}/{INTERIM_FILENAME}"

REPORT_FOLDER = "../reports"

TARGET = "y"
LAG_TARGET = "x_y_lagged"

FIG_WIDTH = 15
FIG_HEIGHT = 3

# DATA

In [4]:
df = pd.read_parquet(RAW_FILEPATH, engine='pyarrow')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6048 entries, 2023-01-16 01:00:00+01:00 to 2023-09-25 00:00:00+02:00
Data columns (total 34 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   x01         6047 non-null   float64
 1   x02         6047 non-null   float64
 2   x03         6047 non-null   float64
 3   x04         6047 non-null   float64
 4   x05         6047 non-null   float64
 5   x06         6047 non-null   float64
 6   x07         6047 non-null   float64
 7   x08         6047 non-null   float64
 8   x09         6047 non-null   float64
 9   x10         6047 non-null   float64
 10  x11         6047 non-null   float64
 11  x12         6047 non-null   float64
 12  x13         6047 non-null   float64
 13  x14         6047 non-null   float64
 14  x15         6047 non-null   float64
 15  x16         6047 non-null   float64
 16  x17         6047 non-null   float64
 17  x18         6047 non-null   float64
 18  x19         6047 non

# SPLIT

As imputation strategy as well as other postprocessing a future engineering shall not leak to the test data, split has to be done 1st.

Because the data represent time series, the split shall take into account time aspect as well.